In [21]:
# main.py

from puzzle import random_selector, ELEMENTS, is_solvable, format_puzzle
from bfs import bfs
from dl_dfs import dl_dfs

def get_initial_state():
    mode = input("Generate a random solvable state (R) or enter a custom state (C)? (R/C): ").upper()
    
    if mode == 'C':
        while True:
            try:
                # Expecting space-separated numbers, e.g., "1 2 3 4 5 6 7 8 0"
                state_str = input("Enter the 9 tile numbers (1-8 and 0 for blank, separated by spaces): ")
                state = tuple(int(x) for x in state_str.split())
                
                if len(state) != 9 or sorted(list(state)) != sorted(ELEMENTS):
                    print("Invalid input. Must be 9 unique numbers (0-8).")
                    continue
                
                if not is_solvable(state):
                    print("This state is NOT solvable. Please enter a solvable state or try another one.")
                    continue
                
                return state
            except ValueError:
                print("Invalid input. Please use numbers only.")
    else:
        # Keep generating random states until a solvable one is found
        while True:
            state = random_selector(ELEMENTS)
            if is_solvable(state):
                print("Generated a random solvable state.")
                return state

def main():
    print("=" * 30)
    print("8 Puzzle Solver")
    print("=" * 30)
    
    # 1. Get Initial State
    init_state = get_initial_state()
    print("\nInitial State")
    print(format_puzzle(init_state))
    
    # 2. Select Algorithm
    while True:
        algo_choice = input("Choose Algorithm: Breadth-First Search (B) or Depth-Limited DFS (D)? (B/D): ").upper()
        if algo_choice in ('B', 'D'):
            break
        print("Invalid choice. Please enter 'B' or 'D'.")
        
    # 3. Run Search
    if algo_choice == 'B':
        bfs(init_state)
    elif algo_choice == 'D':
        while True:
            try:
                limit = int(input("Enter the Depth Limit (e.g., 20): "))
                if limit > 0:
                    break
                print("Limit must be a positive integer.")
            except ValueError:
                print("Invalid input. Please enter a number.")
        dl_dfs(init_state, limit)

if __name__ == "__main__":
    main()

8 Puzzle Solver


Generate a random solvable state (R) or enter a custom state (C)? (R/C):  r


Generated a random solvable state.

Initial State
[5] [6] [4]
[8] [1] [7]
[3] [ ] [2]



Choose Algorithm: Breadth-First Search (B) or Depth-Limited DFS (D)? (B/D):  d
Enter the Depth Limit (e.g., 20):  30


Solution found within depth limit 30! Time taken: 2.3254 seconds

--- Solution Path (DL-DFS (Limit=30)) ---

Step 0 (Start):
[5] [6] [4]
[8] [1] [7]
[3] [ ] [2]


Step 1 (up):
[5] [6] [4]
[8] [ ] [7]
[3] [1] [2]


Step 2 (up):
[5] [ ] [4]
[8] [6] [7]
[3] [1] [2]


Step 3 (left):
[ ] [5] [4]
[8] [6] [7]
[3] [1] [2]


Step 4 (down):
[8] [5] [4]
[ ] [6] [7]
[3] [1] [2]


Step 5 (down):
[8] [5] [4]
[3] [6] [7]
[ ] [1] [2]


Step 6 (right):
[8] [5] [4]
[3] [6] [7]
[1] [ ] [2]


Step 7 (up):
[8] [5] [4]
[3] [ ] [7]
[1] [6] [2]


Step 8 (right):
[8] [5] [4]
[3] [7] [ ]
[1] [6] [2]


Step 9 (down):
[8] [5] [4]
[3] [7] [2]
[1] [6] [ ]


Step 10 (left):
[8] [5] [4]
[3] [7] [2]
[1] [ ] [6]


Step 11 (up):
[8] [5] [4]
[3] [ ] [2]
[1] [7] [6]


Step 12 (left):
[8] [5] [4]
[ ] [3] [2]
[1] [7] [6]


Step 13 (up):
[ ] [5] [4]
[8] [3] [2]
[1] [7] [6]


Step 14 (right):
[5] [ ] [4]
[8] [3] [2]
[1] [7] [6]


Step 15 (right):
[5] [4] [ ]
[8] [3] [2]
[1] [7] [6]


Step 16 (down):
[5] [4] [2]
[8] [3] [ ]
[1

In [7]:
# puzzle.py

import random

# Constants
ROWS, COLS = (3, 3)
BLANK = 0
ELEMENTS = [BLANK, 1, 2, 3, 4, 5, 6, 7, 8]
GOAL_STATE = (1, 2, 3, 4, 5, 6, 7, 8, BLANK)
MOVE_OFFSETS = {
    "up": -3,
    "down": 3,
    "left": -1,
    "right": 1
}
HEADER_1 = "8 Puzzle Solver"
BORDER = "=" * len(HEADER_1)

def random_selector(elements: list, num_elements_to_select: int = 9) -> tuple:
    
    # Randomly chooses elements and determines the initial state tuple.
    
    state_list = []
    
    # Create a mutable copy
    available_elements = elements.copy()

    while (num_elements_to_select > 0):
        selected = random.choice(available_elements)
        available_elements.remove(selected)
        state_list.append(selected)
        num_elements_to_select -= 1

    return tuple(state_list)

def is_solvable(state: tuple) -> bool:
    '''
    Checks if the puzzle state is solvable based on the inversion count.
    '''
    # Remove the blank tile
    arr = [x for x in state if x != BLANK]

    # Calculate the inversion count
    inv_count = 0
    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            if arr[i] > arr[j]:
                inv_count += 1 
    
    # For a 3x3 grid, solvability requires an even inversion count
    return inv_count % 2 == 0

def is_goal_state(state: tuple) -> bool:
    
    # Checks if the current state matches the goal state.
    return GOAL_STATE == state

def get_neighbors(current_state: tuple) -> list[tuple]:
    '''
    Generates all valid neighbor states and the move (direction) required to reach them.
    Returns: [(new_state, move_direction), ...]
    '''
    index = current_state.index(BLANK)
    row = index // COLS
    col = index % COLS
    
    # Determine which moves are valid based on the blank's position
    valid_moves = []
    if row > 0: valid_moves.append("up")
    if row < ROWS - 1: valid_moves.append("down")
    if col > 0: valid_moves.append("left")
    if col < COLS - 1: valid_moves.append("right")

    neighbor_results = []
    for move in valid_moves:
        swap_index = index + MOVE_OFFSETS[move]
        
        # Create a new state list from the tuple
        new_state_list = list(current_state)
        
        # Swap the blank tile with the adjacent tile
        new_state_list[index], new_state_list[swap_index] = new_state_list[swap_index], new_state_list[index]
        
        neighbor_results.append((tuple(new_state_list), move))
    
    return neighbor_results

def get_path(parent: dict, moves: dict, goal_state: tuple) -> tuple[list, list]:
    '''
    Reconstructs the solution path and move sequence from the parent/moves dictionaries.
    '''
    path = []
    move_arr = []
    state = goal_state

    # Trace back from the goal state to the root (parent=None)
    while state is not None:
        path.append(state)
        move_arr.append(moves.get(state)) # Use .get for root state (None)
        state = parent.get(state)
    
    path.reverse()
    move_arr.reverse()

    return path, move_arr

def format_puzzle(state: tuple) -> str:
    '''
    Formats the 1D state tuple into a readable 3x3 grid string.
    '''
    grid_str = ""
    for i in range(0, len(state), COLS):
        row = state[i:i + COLS]
        row_str = " ".join(f"[{' ' if tile == BLANK else tile}]" for tile in row)
        grid_str += row_str + "\n"
    return grid_str

def print_solution(path: list, move_arr: list, algorithm: str):
    '''
    Prints the step-by-step solution path.
    '''
    print(f"\n--- Solution Path ({algorithm}) ---")
    for i, state in enumerate(path):
        current_move = move_arr[i] if i > 0 else "Start"
        print(f"\nStep {i} ({current_move}):")
        print(format_puzzle(state))
    print(f"Total Moves: {len(path) - 1}")

In [ ]:
# dl_dfs.py

import time
from puzzle import is_solvable, is_goal_state, get_neighbors, get_path, print_solution

def recursive_dfs(state, goal, limit, path, parent, moves):
    '''
    Recursive helper function for DL-DFS.
    Returns: goal_state_tuple if solution found, else None
    '''
    if is_goal_state(state):
        return state
    
    # If the depth limit is reached, stop searching along this path
    if limit == 0:
        return None
    
    # Explore all possible neighbors
    for next_state, move in get_neighbors(state):
        
        # Check if the next state is not already in the current path 
        # (This is important for DFS to avoid immediate cycles and repeated states on the path)
        if next_state not in path:
            # Update parent/moves for path reconstruction upon success
            parent[next_state] = state
            moves[next_state] = move

            # Recurse with reduced limit and extended path
            result = recursive_dfs(next_state, goal, limit - 1, path + (next_state,), parent, moves)
            
            # If a solution is found in the recursive call, pass it up
            if result is not None:
                return result

    return None

def dl_dfs(init_state: tuple, limit: int):
    '''
    Implements the Depth-Limited Depth-First Search algorithm.
    '''
    start = time.perf_counter()

    if not is_solvable(init_state):
        print("No solution exists. Start state is of odd parity.")
        return None
    
    # DL-DFS uses a path check instead of a global visited set.
    # We pass parent/moves dictionaries to the recursive function to track the solution path.
    parent = {init_state: None}
    moves = {init_state: None}

    # Start the recursive search
    goal_state_found = recursive_dfs(init_state, limit, limit, (init_state,), parent, moves)

    if goal_state_found:
        end = time.perf_counter()
        print(f"Solution found within depth limit {limit}! Time taken: {end - start:.4f} seconds")
        path, move_arr = get_path(parent, moves, goal_state_found)
        print_solution(path, move_arr, f"DL-DFS (Limit={limit})")
        return path
    else:
        end = time.perf_counter()
        print(f"\nNo solution found within the depth limit of {limit}. Time taken: {end - start:.4f} seconds\n")
        return None

In [9]:
# bfs.py

from collections import deque
import time
from puzzle import is_solvable, is_goal_state, get_neighbors, get_path, print_solution

def bfs(init_state: tuple):
    '''
    Implements the Breadth-First Search algorithm for the 8-puzzle.
    '''
    start = time.perf_counter()
    
    if not is_solvable(init_state):
        print("No solution exists. Start state is of odd parity.")
        return None

    # Data Structures for BFS
    root_node = init_state
    parent = {root_node: None} # Tracks the state from which the current state was reached
    moves = {root_node: None}  # Tracks the move (up/down/etc.) used to reach the state
    visited = {root_node}      # Tracks all states already explored
    bfs_queue = deque([root_node])
    iterations = 0

    while bfs_queue:
        current_state = bfs_queue.popleft()
        iterations += 1

        if is_goal_state(current_state):
            end = time.perf_counter()
            print(f"Solution found at iteration {iterations}! Time taken: {end - start:.4f} seconds")
            path, move_arr = get_path(parent, moves, current_state)
            print_solution(path, move_arr, "BFS")
            return path
        
        for new_state, move in get_neighbors(current_state):
            if new_state not in visited:
                visited.add(new_state)
                moves[new_state] = move
                parent[new_state] = current_state
                bfs_queue.append(new_state)
    
    # Should not be reached if the puzzle is solvable and GOAL_STATE is correct
    print("Search failed (This should only happen if puzzle is unsolvable or goal state is unreachable).")
    return None